In [12]:
import os 
from dotenv import load_dotenv
load_dotenv()


python-dotenv could not parse statement starting at line 6


True

In [13]:
from langchain_groq import ChatGroq
api_key=os.getenv("GROQ_API_KEY")
llm=ChatGroq(groq_api_key=api_key,model="llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000027143D3C910>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000027143D3D0C0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [14]:
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [15]:
speech="""

Today, as we stand at the crossroads of history, our nation is moving forward with confidence, courage, and commitment. India is not merely a country — it is an idea, a vision, and a civilization that has inspired humanity for thousands of years.

In the past decade, we have witnessed a transformation that touches every Indian. From the smallest village to the largest city, from farmers to entrepreneurs, from students to scientists — a new India is rising, built on the foundations of innovation, inclusion, and integrity.



"""

In [16]:
speech

'\n\nToday, as we stand at the crossroads of history, our nation is moving forward with confidence, courage, and commitment. India is not merely a country — it is an idea, a vision, and a civilization that has inspired humanity for thousands of years.\n\nIn the past decade, we have witnessed a transformation that touches every Indian. From the smallest village to the largest city, from farmers to entrepreneurs, from students to scientists — a new India is rising, built on the foundations of innovation, inclusion, and integrity.\n\n\n\n'

In [17]:
chat_message=[
    SystemMessage(content="You are a helpful assistant that summarizes text"),
    HumanMessage(content=f"please provide all the details of this:/n Text:{speech}")
]

In [18]:
llm.get_num_tokens(speech)

110

In [19]:
llm(chat_message).content

"Here's a summary of the given text:\n\n**Key Points:**\n\n1. India is standing at a significant moment in history, moving forward with confidence, courage, and commitment.\n2. India is not just a country, but an idea, vision, and civilization that has inspired humanity for thousands of years.\n3. Over the past decade, India has undergone a transformative period, impacting every Indian.\n4. This transformation has touched various aspects of Indian society, including villages, cities, farmers, entrepreneurs, students, and scientists.\n5. The new India is built on the foundations of innovation, inclusion, and integrity.\n\n**Main Themes:**\n\n1. National Progress\n2. Indian Identity\n3. Transformation and Growth\n\n**Overall Message:**\nThe text portrays a positive and optimistic view of India's current state and future prospects, emphasizing its rich history, cultural significance, and potential for growth and development."

In [20]:
###prompt template text summarization
from langchain.chains import LLMChain
from langchain import PromptTemplate
generictemplate=""" 
write a summary of the following speech:
speech:{speech}
Translate the precise summary to {language}
"""
prompt=PromptTemplate(
    input_variables=['speech','language'],
    template=generictemplate
)
prompt




PromptTemplate(input_variables=['language', 'speech'], input_types={}, partial_variables={}, template=' \nwrite a summary of the following speech:\nspeech:{speech}\nTranslate the precise summary to {language}\n')

In [21]:
complete_prompt=prompt.format(speech=speech,language="French")
complete_prompt


' \nwrite a summary of the following speech:\nspeech:\n\nToday, as we stand at the crossroads of history, our nation is moving forward with confidence, courage, and commitment. India is not merely a country — it is an idea, a vision, and a civilization that has inspired humanity for thousands of years.\n\nIn the past decade, we have witnessed a transformation that touches every Indian. From the smallest village to the largest city, from farmers to entrepreneurs, from students to scientists — a new India is rising, built on the foundations of innovation, inclusion, and integrity.\n\n\n\n\nTranslate the precise summary to French\n'

In [22]:
llm.get_num_tokens(complete_prompt)

132

In [26]:
llm_chain=LLMChain(llm=llm,prompt=prompt)
summary=llm_chain.run({'speech':speech,'language':'hindi'})

In [27]:
summary

'Samajhsheen hai ki hum aaj itihaas ki rah par khada hain. hamari desh ne saavdhaanee, sainyashakti aur samarpitata ke saath aage badha hai. Bharat na sirf ek desh hai, balki yah ek vichar, ek drishti aur ek sanskriti hai jo hazaar varshon se manusya ko prernit karti aayi hai.\n\nIn sabhi varshon mein, humne ek anubhav dekha hai jo har hindustani ko prabhavit kar raha hai. se chhote gaon se lekar sabse bade shaharon tak, kisan se lekar udyogpatiyon tak, vidyarthis se lekar viganon tak - ek naya Bharat uth raha hai, jiska aadhaar navaankarshna, shamilta aur avishvas par aadharit hai.'

In [29]:
##stuff document chain
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("apjspeech.pdf")
docs=loader.load_and_split()
docs



[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealt

In [31]:
template="""write the concise and short summary of 
 following speech,speech:{text}
 """
prompt=PromptTemplate(
    input_variables=['text'],
    template=template
)

In [32]:
from langchain.chains.summarize import load_summarize_chain


In [33]:
chain=load_summarize_chain(llm,chain_type="stuff",prompt=prompt,verbose=True)
chain.run(docs)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
write the concise and short summary of 
 following speech,speech:A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online inter

'Here\'s a concise and short summary of Dr. A.P.J. Abdul Kalam\'s departing speech as the President of India:\n\nDr. Kalam expressed his gratitude to the nation for the opportunity to serve as President for five years. He highlighted the importance of empowering villages, mobilizing rural core competence for competitiveness, and promoting agricultural growth through the "Seed to Food" initiative. He also emphasized the need to defeat problems and succeed through partnerships and courageous efforts.\n\nDr. Kalam mentioned his experiences with various initiatives, including the Pan African e-Network, which connected 53 African countries through a seamless and integrated satellite, fiber optics, and wireless network. He also highlighted the importance of defending the nation and promoting a youth movement for a developed India 2020.\n\nIn his conclusion, Dr. Kalam outlined a distinctive profile for a developed India 2020, which includes:\n\n1. Reduced rural-urban divide\n2. Equitable dist

In [34]:
##map reduce 
from langchain.text_splitter import RecursiveCharacterTextSplitter  

In [35]:
docs

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealt

In [37]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=100).split_documents(docs)

text_splitter


[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealt

In [38]:
len(text_splitter)

13

In [40]:
chunk_prompt="""
Please summarize the below text speeh:
speech:'{text}'
Summary:
"""
map_prompt_template=PromptTemplate(
    input_variables=['text'],
    template=chunk_prompt
)

In [ ]:
final_prompt='''
provide the final summary of entire speech with these important points
Add a motivation title,start the precise the summary of the speech .speech:{text}

'''
final_prompt_template=PromptTemplate(
    input_variables=['text'],
    template=final_prompt
    
)

In [42]:
summary_chain=load_summarize_chain(llm,chain_type="map_reduce",map_prompt=map_prompt_template,combine_prompt=final_prompt_template,verbose=True)

output=summary_chain.run(text_splitter)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Please summarize the below text speeh:
speech:'A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I ha

Token indices sequence length is longer than the specified maximum sequence length for this model (1976 > 1024). Running this sequence through the model will result in indexing errors




> Entering new LLMChain chain...
Prompt after formatting:

provide the final summary of entire speech with these important points
Add a motivation title,start the precise the summary of the speech .speech:The speech is by A.P.J. Abdul Kalam, the former President of India, on his departure after five years in office. He expresses gratitude to the people of India and those living abroad for their association and support. 

Kalam emphasizes ten key messages from his experiences during his tenure, including:

1. Accelerating development to fulfill the aspirations of the youth
2. Empowering villages
3. Mobilizing rural core competence for competitiveness
4. Focusing on agriculture from seed to food
5. Defeating problems and succeeding
6. Overcoming problems through partnership
7. Showing courage in the face of calamities
8. Fostering connectivity for societal transformation
9. Defending the nation
10. Launching a youth movement for a Developed India by 2020

He recalls a question from a y

In [43]:
##refine chain summarization
chain=load_summarize_chain(llm,chain_type="refine",verbose=True)
output_summary=chain.run(text_splitter)
output_summary




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have man

'Refined Summary:\n\nA.P.J. Abdul Kalam\'s departing speech from the Rashtrapati Bhavan in 2007 highlights his gratitude to the people of India and the various groups that supported him during his five-year tenure. He shares 10 key messages to accelerate the country\'s development, including:\n\n1. Empowering villages and rural areas\n2. Mobilizing rural core competence for competitiveness\n3. Focusing on sustainable agricultural growth through a "Seed to Food" approach\n4. Defeating problems and succeeding\n5. Overcoming challenges through partnerships\n6. Demonstrating courage in the face of calamities\n7. Fostering connectivity for societal transformation\n8. Defending the nation\n9. Harnessing the potential of youth for a developed India\n10. Accelerating development through the aspirations of the youth\n\nDuring his speech, he shares a personal anecdote about a young girl, Anukriti, who asked him why India couldn\'t become a developed nation before 2020. This question reflects the